In [1]:
from multiprocessing import Process, Pipe
from libs.hq.hq import *
from libs.multiprocessing.assist import assist
from libs.db.odm import ZhangTingStock, LiangbiStock, NewStock
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.events import EVENT_JOB_ERROR, EVENT_JOB_EXECUTED, EVENT_JOB_MISSED
from mongoengine import connect
import numpy as np
import pandas as pd
HQ.init('C:\通达信\通达信_通赢版_7.35-5.90-17061313_健谈')

In [2]:
time_0900 = datetime.now().replace(hour=9, minute=0, second=0, microsecond=0)
time_0905 = datetime.now().replace(hour=9, minute=5, second=0, microsecond=0)
time_0925 = datetime.now().replace(hour=9, minute=25, second=0, microsecond=0)
time_0930 = datetime.now().replace(hour=9, minute=30, second=0, microsecond=0)
time_1130 = datetime.now().replace(hour=11, minute=30, second=16, microsecond=0)
time_1300 = datetime.now().replace(hour=13, minute=0, second=0, microsecond=0)
time_1500 = datetime.now().replace(hour=15, minute=0, second=16, microsecond=0)

(a_c, b_c)=Pipe()
assist_p = Process(target=assist, args=(a_c,))
assist_p.start()

check_snapshot_jobs = {}

def save():
    b_c.send({'close_hdf5_file': ''})
    HQ.save_hdf5()
    default_hdf5 = './hdf5/default.hdf5'
    HQ.save_hdf5(default_hdf5)
    assist_p.terminate()

def job_listener(event):
    while b_c.poll():
        msg = b_c.recv()
        print('message from assist process: ', str(msg))

    now = datetime.now()
    if event.code == EVENT_JOB_ERROR:
        print('The job crashed :(')
    elif event.code == EVENT_JOB_MISSED:
        print(now.strftime('%Y-%m-%d %H:%M:%S'), 'job was missed', event.job_id)
    elif event.code == EVENT_JOB_EXECUTED:
        if event.job_id in check_snapshot_jobs:
            
            dt = event.retval['time']
            idx_sc = HQ.sc.get_index(dt)
            view = HQ.sc.get_view(dt)
            b_c.send({'update_sc': {'idx': idx_sc, 'view': view}})
            
            ztIndices = event.retval['ztIndices']
            for idx in ztIndices:
                code = HQ.codes[idx]
                stock = ZhangTingStock()
                stock.code = code
                stock.name = HQ.stocks[code]['name']
                stock.time = dt
                stock.save()

            lbIndices = event.retval['lbIndices']
            for idx in lbIndices:
                code = HQ.codes[idx]
                stock = LiangbiStock()
                stock.code = code
                stock.name = HQ.stocks[code]['name']
                stock.time = dt
                stock.save()

            print('done, job:', check_snapshot_jobs[event.job_id].name, end='')
            print(', used: ', (now - dt).total_seconds(), 's')

In [3]:
scheduler = BackgroundScheduler()

now = datetime.now()
default_hdf5 = './hdf5/default.hdf5'

if HQ.is_holiday_today():
    filename = default_hdf5
    HQ.load_hdf5(filename)
elif now < time_0900:
    filename = default_hdf5
    HQ.load_hdf5(filename)
    
    scheduler.add_job(HQ.initHQ, 'date', run_date=time_0905.strftime('%Y-%m-%d %H:%M:%S'))
    
    job1 = scheduler.add_job(HQ.check_snapshot, 'interval', 
                          start_date=time_0925.strftime('%Y-%m-%d %H:%M:%S'), 
                          end_date=time_1130.strftime('%Y-%m-%d %H:%M:%S'), 
                          seconds=3)
    check_snapshot_jobs[job1.id] = job1
    job2 = scheduler.add_job(HQ.check_snapshot, 'interval', 
                          start_date=time_1300.strftime('%Y-%m-%d %H:%M:%S'), 
                          end_date=time_1500.strftime('%Y-%m-%d %H:%M:%S'), 
                          seconds=3)
    check_snapshot_jobs[job2.id] = job2

    scheduler.add_job(save, 'date', run_date=time_1500.strftime('%Y-%m-%d %H:%M:%S'))

elif time_0900 <= now < time_1500:
    HQ.initHQ()
    
    job1 = scheduler.add_job(HQ.check_snapshot, 'interval', 
                          start_date=time_0925.strftime('%Y-%m-%d %H:%M:%S'), 
                          end_date=time_1130.strftime('%Y-%m-%d %H:%M:%S'), 
                          seconds=3)
    check_snapshot_jobs[job1.id] = job1
    job2 = scheduler.add_job(HQ.check_snapshot, 'interval', 
                          start_date=time_1300.strftime('%Y-%m-%d %H:%M:%S'), 
                          end_date=time_1500.strftime('%Y-%m-%d %H:%M:%S'), 
                          seconds=3)
    check_snapshot_jobs[job2.id] = job2
    
    scheduler.add_job(save, 'date', run_date=time_1500.strftime('%Y-%m-%d %H:%M:%S'))
else:
    filename = default_hdf5
    HQ.load_hdf5(filename)

scheduler.add_listener(job_listener, EVENT_JOB_EXECUTED | EVENT_JOB_ERROR | EVENT_JOB_MISSED)

scheduler.start()

In [4]:
len(HQ.codes)

3411

3413 200 17.065
200:200/200 | 400:200/200 | 600:200/200 | 800:200/200 | 1000:200/200 | 1200:200/200 | 1400:200/200 | 1600:200/200 | 1800:200/200 | 2000:200/200 | 2200:200/200 | 2400:200/200 | 2600:200/200 | 2800:200/200 | 3000:200/200 | 3200:200/200 | 3400:200/200 | 3413:13/200 | 
0  failed codes:  set()
try to refetch: ... ... [Getting data:]############################################################文件 [ ./hdf5/default.hdf5 ] 已经存在，将被删除 ... ... 已被删除
行情 已经 写入文件： ./hdf5/default.hdf5
2017-11-02 09:25:00  : check snapshot  -->>  fc[ 0 ] sc[ 0 ]
2017-11-02 09:25:00.175688 job was executed successfully HQ.check_snapshot 2017-11-02 09:25:00.004952
2017-11-02 09:25:03  : check snapshot  -->>  fc[ 0 ] sc[ 0 ]
2017-11-02 09:25:03.171511 job was executed successfully HQ.check_snapshot 2017-11-02 09:25:03.001861
2017-11-02 09:25:06  : check snapshot  -->>  fc[ 0 ] sc[ 0 ]
2017-11-02 09:25:06.247204 job was executed successfully HQ.check_snapshot 2017-11-02 09:25:06.005421
2017-11-02 09:25:09  : c

2017-11-02 09:27:21.214456 job was executed successfully HQ.check_snapshot 2017-11-02 09:27:21.000737
2017-11-02 09:27:24  : check snapshot  -->>  fc[ 0 ] sc[ 0 ]
2017-11-02 09:27:24.218912 job was executed successfully HQ.check_snapshot 2017-11-02 09:27:24.001334
2017-11-02 09:27:27  : check snapshot  -->>  fc[ 0 ] sc[ 0 ]
2017-11-02 09:27:27.208168 job was executed successfully HQ.check_snapshot 2017-11-02 09:27:27.001219
2017-11-02 09:27:30  : check snapshot  -->>  fc[ 0 ] sc[ 0 ]
2017-11-02 09:27:30.207155 job was executed successfully HQ.check_snapshot 2017-11-02 09:27:30.000483
2017-11-02 09:27:33  : check snapshot  -->>  fc[ 0 ] sc[ 0 ]
2017-11-02 09:27:33.215179 job was executed successfully HQ.check_snapshot 2017-11-02 09:27:33.000502
2017-11-02 09:27:36  : check snapshot  -->>  fc[ 0 ] sc[ 0 ]
2017-11-02 09:27:36.205830 job was executed successfully HQ.check_snapshot 2017-11-02 09:27:36.000616
2017-11-02 09:27:39  : check snapshot  -->>  fc[ 0 ] sc[ 0 ]
2017-11-02 09:27:39.21

2017-11-02 09:29:54.211104 job was executed successfully HQ.check_snapshot 2017-11-02 09:29:54.000735
2017-11-02 09:29:57  : check snapshot  -->>  fc[ 0 ] sc[ 0 ]
2017-11-02 09:29:57.210296 job was executed successfully HQ.check_snapshot 2017-11-02 09:29:57.001008
2017-11-02 09:30:00  : check snapshot  -->>  fc[ 0 ] sc[ 0 ]
2017-11-02 09:30:00.246415 job was executed successfully HQ.check_snapshot 2017-11-02 09:30:00.000668
2017-11-02 09:30:03  : check snapshot  -->>  fc[ 0 ] sc[ 1 ]
2017-11-02 09:30:03.218100 job was executed successfully HQ.check_snapshot 2017-11-02 09:30:03.001211
2017-11-02 09:30:06  : check snapshot  -->>  fc[ 0 ] sc[ 2 ]
2017-11-02 09:30:06.224987 job was executed successfully HQ.check_snapshot 2017-11-02 09:30:06.000325
2017-11-02 09:30:09  : check snapshot  -->>  fc[ 0 ] sc[ 3 ]
2017-11-02 09:30:09.209384 job was executed successfully HQ.check_snapshot 2017-11-02 09:30:09.000367
2017-11-02 09:30:12  : check snapshot  -->>  fc[ 0 ] sc[ 4 ]
2017-11-02 09:30:12.20

2017-11-02 09:32:27  : check snapshot  -->>  fc[ 2 ] sc[ 49 ]
2017-11-02 09:32:27.203518 job was executed successfully HQ.check_snapshot 2017-11-02 09:32:27.000746
2017-11-02 09:32:30  : check snapshot  -->>  fc[ 2 ] sc[ 50 ]
2017-11-02 09:32:30.217212 job was executed successfully HQ.check_snapshot 2017-11-02 09:32:30.001485
2017-11-02 09:32:33  : check snapshot  -->>  fc[ 2 ] sc[ 51 ]
2017-11-02 09:32:33.217029 job was executed successfully HQ.check_snapshot 2017-11-02 09:32:33.001297
2017-11-02 09:32:36  : check snapshot  -->>  fc[ 2 ] sc[ 52 ]
2017-11-02 09:32:36.222099 job was executed successfully HQ.check_snapshot 2017-11-02 09:32:36.000906
2017-11-02 09:32:39  : check snapshot  -->>  fc[ 2 ] sc[ 53 ]
2017-11-02 09:32:39.215833 job was executed successfully HQ.check_snapshot 2017-11-02 09:32:39.000488
2017-11-02 09:32:42  : check snapshot  -->>  fc[ 2 ] sc[ 54 ]
2017-11-02 09:32:42.204215 job was executed successfully HQ.check_snapshot 2017-11-02 09:32:42.000867
2017-11-02 09:32

...

2017-11-02 14:48:33  : check snapshot  -->>  fc[ 229 ] sc[ 4577 ]
2017-11-02 14:48:33.209686 job was executed successfully HQ.check_snapshot 2017-11-02 14:48:33.001309
2017-11-02 14:48:36  : check snapshot  -->>  fc[ 229 ] sc[ 4578 ]
2017-11-02 14:48:36.217309 job was executed successfully HQ.check_snapshot 2017-11-02 14:48:36.001003
2017-11-02 14:48:39  : check snapshot  -->>  fc[ 229 ] sc[ 4579 ]
2017-11-02 14:48:39.229829 job was executed successfully HQ.check_snapshot 2017-11-02 14:48:39.000923
2017-11-02 14:48:42  : check snapshot  -->>  fc[ 229 ] sc[ 4580 ]
2017-11-02 14:48:42.222549 job was executed successfully HQ.check_snapshot 2017-11-02 14:48:42.001333
2017-11-02 14:48:45  : check snapshot  -->>  fc[ 229 ] sc[ 4581 ]
2017-11-02 14:48:45.203292 job was executed successfully HQ.check_snapshot 2017-11-02 14:48:45.000927
2017-11-02 14:48:48  : check snapshot  -->>  fc[ 229 ] sc[ 4582 ]
2017-11-02 14:48:48.241025 job was executed successfully HQ.check_snapshot 2017-11-02 14:48:48

2017-11-02 14:51:00  : check snapshot  -->>  fc[ 231 ] sc[ 4626 ]
2017-11-02 14:51:00.217687 job was executed successfully HQ.check_snapshot 2017-11-02 14:51:00.000851
2017-11-02 14:51:03  : check snapshot  -->>  fc[ 232 ] sc[ 4627 ]
2017-11-02 14:51:03.225597 job was executed successfully HQ.check_snapshot 2017-11-02 14:51:03.000916
2017-11-02 14:51:06  : check snapshot  -->>  fc[ 232 ] sc[ 4628 ]
2017-11-02 14:51:06.216774 job was executed successfully HQ.check_snapshot 2017-11-02 14:51:06.001057
2017-11-02 14:51:09  : check snapshot  -->>  fc[ 232 ] sc[ 4629 ]
2017-11-02 14:51:09.223227 job was executed successfully HQ.check_snapshot 2017-11-02 14:51:09.000180
2017-11-02 14:51:12  : check snapshot  -->>  fc[ 232 ] sc[ 4630 ]
2017-11-02 14:51:12.219500 job was executed successfully HQ.check_snapshot 2017-11-02 14:51:12.001013
2017-11-02 14:51:15  : check snapshot  -->>  fc[ 232 ] sc[ 4631 ]
2017-11-02 14:51:15.224943 job was executed successfully HQ.check_snapshot 2017-11-02 14:51:15

2017-11-02 14:53:27  : check snapshot  -->>  fc[ 234 ] sc[ 4675 ]
2017-11-02 14:53:27.225605 job was executed successfully HQ.check_snapshot 2017-11-02 14:53:27.001199
2017-11-02 14:53:30  : check snapshot  -->>  fc[ 234 ] sc[ 4676 ]
2017-11-02 14:53:30.221916 job was executed successfully HQ.check_snapshot 2017-11-02 14:53:30.000676
2017-11-02 14:53:33  : check snapshot  -->>  fc[ 234 ] sc[ 4677 ]
2017-11-02 14:53:33.226490 job was executed successfully HQ.check_snapshot 2017-11-02 14:53:33.000786
2017-11-02 14:53:36  : check snapshot  -->>  fc[ 234 ] sc[ 4678 ]
2017-11-02 14:53:36.225282 job was executed successfully HQ.check_snapshot 2017-11-02 14:53:36.001032
2017-11-02 14:53:39  : check snapshot  -->>  fc[ 234 ] sc[ 4679 ]
2017-11-02 14:53:39.223108 job was executed successfully HQ.check_snapshot 2017-11-02 14:53:39.000354
2017-11-02 14:53:42  : check snapshot  -->>  fc[ 234 ] sc[ 4680 ]
2017-11-02 14:53:42.213910 job was executed successfully HQ.check_snapshot 2017-11-02 14:53:42

2017-11-02 14:55:54  : check snapshot  -->>  fc[ 236 ] sc[ 4724 ]
2017-11-02 14:55:54.226949 job was executed successfully HQ.check_snapshot 2017-11-02 14:55:54.000755
2017-11-02 14:55:57  : check snapshot  -->>  fc[ 236 ] sc[ 4725 ]
2017-11-02 14:55:57.233713 job was executed successfully HQ.check_snapshot 2017-11-02 14:55:57.001626
2017-11-02 14:56:00  : check snapshot  -->>  fc[ 236 ] sc[ 4726 ]
2017-11-02 14:56:00.206769 job was executed successfully HQ.check_snapshot 2017-11-02 14:56:00.001232
2017-11-02 14:56:03  : check snapshot  -->>  fc[ 237 ] sc[ 4727 ]
2017-11-02 14:56:03.213177 job was executed successfully HQ.check_snapshot 2017-11-02 14:56:03.000858
2017-11-02 14:56:06  : check snapshot  -->>  fc[ 237 ] sc[ 4728 ]
2017-11-02 14:56:06.219206 job was executed successfully HQ.check_snapshot 2017-11-02 14:56:06.001424
2017-11-02 14:56:09  : check snapshot  -->>  fc[ 237 ] sc[ 4729 ]
2017-11-02 14:56:09.247035 job was executed successfully HQ.check_snapshot 2017-11-02 14:56:09

2017-11-02 14:58:21  : check snapshot  -->>  fc[ 239 ] sc[ 4773 ]
2017-11-02 14:58:21.222444 job was executed successfully HQ.check_snapshot 2017-11-02 14:58:21.000955
2017-11-02 14:58:24  : check snapshot  -->>  fc[ 239 ] sc[ 4774 ]
2017-11-02 14:58:24.214152 job was executed successfully HQ.check_snapshot 2017-11-02 14:58:24.001400
2017-11-02 14:58:27  : check snapshot  -->>  fc[ 239 ] sc[ 4775 ]
2017-11-02 14:58:27.221975 job was executed successfully HQ.check_snapshot 2017-11-02 14:58:27.000683
2017-11-02 14:58:30  : check snapshot  -->>  fc[ 239 ] sc[ 4776 ]
2017-11-02 14:58:30.198806 job was executed successfully HQ.check_snapshot 2017-11-02 14:58:30.001474
2017-11-02 14:58:33  : check snapshot  -->>  fc[ 239 ] sc[ 4777 ]
2017-11-02 14:58:33.190212 job was executed successfully HQ.check_snapshot 2017-11-02 14:58:33.001354
2017-11-02 14:58:36  : check snapshot  -->>  fc[ 239 ] sc[ 4778 ]
2017-11-02 14:58:36.214169 job was executed successfully HQ.check_snapshot 2017-11-02 14:58:36

In [ ]:
HQ.save_hdf5()

In [ ]:
for idx in HQ.new_indices:
    t = HQ.new_indices[idx]
    HQ.new_indices[idx] = datetime.strptime(t, '%Y-%m-%d %H:%M:%S')

In [5]:
b_c.send({'close_hdf5_file': ''})
# HQ.save_hdf5()
default_hdf5 = './hdf5/default.hdf5'
HQ.save_hdf5(default_hdf5)
assist_p.terminate()

文件 [ ./hdf5/default.hdf5 ] 已经存在，将被删除 ... ... 已被删除
行情 已经 写入文件： ./hdf5/default.hdf5


In [ ]:
assist_p.terminate()

In [ ]:
assist_p.terminate()
scheduler.shutdown()

In [ ]:
default_hdf5 = './hdf5/default.hdf5'
HQ.load_hdf5(default_hdf5)

In [ ]:
default_hdf5 = './hdf5/default.hdf5'
HQ.save_hdf5(default_hdf5)

In [ ]:
HQ.sc.data[6]

In [ ]:
default_hdf5 = './hdf5/default.hdf5'
HQ.load_hdf5(default_hdf5)

scheduler = BackgroundScheduler()
job = scheduler.add_job(HQ.check_snapshot, 'interval', 
                          start_date='2017-10-30 23:00:00', 
                          end_date='2017-10-30 23:59:00', 
                          seconds=3)
check_snapshot_jobs[job.id] = job
scheduler.add_listener(job_listener, EVENT_JOB_EXECUTED | EVENT_JOB_ERROR | EVENT_JOB_MISSED)
scheduler.start()
print('job_name:', job.name)

In [ ]:
assist_p.terminate()
scheduler.shutdown()

In [ ]:
scheduler.get_jobs()

In [ ]:
hqstocks2 = HQ.stocks
hqcodes2 = HQ.codes
hqbankuai2 = HQ.bankuai
%store hqstocks2
%store hqcodes2
%store hqbankuai2

In [ ]:
HQ.init('C:\通达信\通达信_通赢版_7.35-5.90-17061313_健谈')
%store -r d2a_stock_info
%store -r hqcodes
%store -r hqstocks
%store -r fcdata
%store -r scdata

In [ ]:
kwargs = dict(
    codes = hqcodes,
    stocks = hqstocks,
    bankuai = {},
    
    d2a_stock_info=d2a_stock_info,
    
    zt_indices = {},
    lb_indices = {},
    zs_indices = [],
    
    new_stock_indices = {}
    
#     fcdata = fcdata,
    fcdata = np.empty((242, len(hqcodes), len(HQ.cols)), dtype=np.float32),
    scdata = scdata,
)
HQ.setup(**kwargs)
HQ.sc.data[:,:,7:] = np.nan
HQ.fc.data[:,:,:] = np.nan

In [ ]:
dt1 = datetime.now().replace(hour=9, minute=25, second=0, microsecond=0)
dt2 = datetime.now().replace(hour=13, minute=0, second=0, microsecond=0)
for i in range(2505):
    HQ.check_snapshot((dt1 + timedelta(seconds=3*i)))
for i in range(2405):
    HQ.check_snapshot((dt2 + timedelta(seconds=3*i)))

In [ ]:
dt = datetime.now().replace(hour=11, minute=29, second=0, microsecond=0)
HQ.check_snapshot(dt)
print(HQ.fc.get_index(dt-timedelta(seconds=1)), HQ.sc.get_index(dt))
HQ.fc.get_view(dt-timedelta(seconds=1))[3174]

In [ ]:
HQ.sc.data[2380:2399,3174,5]

In [ ]:
hqdt_2 = datetime.now().replace(hour=9, minute=0, second=0, microsecond=0)

hqcodes_2 = HQ.codes
hqstocks_2 = HQ.stocks
hqbankuai_2 = HQ.bankuai

d2a_stock_info_data_2 = HQ.d2a_stock_info.data

zt_indices_2 = HQ.zt_indices
lb_indices_2 = HQ.lb_indices
new_indices_2 = HQ.new_indices

fcdata_2 = HQ.fc.data
scdata_2 = HQ.sc.data

%store hqdt_2

%store hqcodes_2
%store hqstocks_2
%store hqbankuai_2

%store d2a_stock_info_data_2

%store zt_indices_2
%store lb_indices_2
%store new_indices_2

%store fcdata_2
%store scdata_2

In [ ]:
d2a_stock_info_data_2

In [ ]:
df = pd.DataFrame(data=HQ.fc.data[1], index=HQ.codes, columns=HQ.cols)
df.head(10)

In [ ]:
df2 = pd.DataFrame(data=scdata[21], index=HQ.codes, columns=HQ.cols)
df2.head(10)

In [ ]:
HQ.lb_indices

In [ ]:
ztIndices = HQ.zt_indices
newIndices = HQ.new_indices
lbIndices = HQ.lb_indices
%store ztIndices
%store newIndices
%store lbIndices

In [ ]:
%store

In [ ]:
HQ.save_hdf5()

In [ ]:
HQ.sc.data[5]

In [ ]:
for code in HQ.stocks:
    name = HQ.stocks[code]['name']
    if name.startswith('N'):
        print(code, name)

In [8]:
import plotly
plotly.offline.init_notebook_mode(connected=True)

In [9]:
code = '002796'
code_idx = HQ.codes.index(code)
print(code_idx)
HQ.fc.plot_fenshi(code_idx)

1246
This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]



In [ ]:
def plot_fenshi(code_idx, fszb='liangbi'):

        high = np.nanmax(data[:, code_idx, 2])
        low = np.nanmin(data[:, code_idx, 2])
        close = data[0, code_idx, 1]
        maxv = max(high, close)
        minv = min(low, close)
        maxv_p = round((maxv - close) / close * 100, 2)
        minv_p = round((minv - close) / close * 100, 2)

        to = data[:, code_idx, 5]
        ta = np.roll(to, 1)
        ta[0] = 0
        turnover = to - ta

        bar_to = go.Bar(
            x=list(range(242)),
            y=turnover,
#             y=data[:, :, -1][:, code_idx],
            name='成交量',
            # hoverinfo='none',
            showlegend=False,
            width=1,
            # line=dict(width=1)
        )

        trace_p = go.Scatter(
            x=list(range(242)),
            y=data[:, :, 2][:, code_idx],
            name='股价',
            # hoverinfo='none',
            showlegend=False,
            # yaxis='y',
            line=dict(
                width=1,
            )
        )
        trace_jj = go.Scatter(
            x=list(range(242)),
            y=data[:, :, 8][:, code_idx],
            name='均价',
            hoverinfo='none',
            showlegend=False,
            line=dict(width=1),
        )
        trace_close = go.Scatter(
            x=list(range(242)),
            y=[close for i in range(242)],
            name='昨收',
            hoverinfo='skip',
            showlegend=False,
            line=dict(
                color=('rgb(22, 96, 167)'),
                width=1,
                dash='dot'),
        )
        if fszb == 'zhenfu_5m':
            idx_zb = 12
            name_zb = '5F振幅'
        elif fszb == 'liangbi':
            idx_zb = 9
            name_zb = '量比'
        else:
            idx_zb = 9
            name_zb = '量比'
        trace_fszb = go.Scatter(
            x=list(range(242)),
            y=data[:, :, idx_zb][:, code_idx],
            name=name_zb,
            # hoverinfo='none',
            showlegend=False,
            line=dict(
                width=1,
            )
        )
        yaxis1 = dict(
            visible=True,
            # fixedrange=True,
            spikethickness=1,
            spikedash='dash',
            spikemode='across',
            zeroline=False,
            # ticklen=1,
            domain=[0, 0.25]
        )
        yaxis2 = dict(
            visible=True,
            # fixedrange=True,
            spikethickness=1,
            spikedash='dash',
            spikemode='across',
            zeroline=False,
            # ticklen=1,
            domain=[0.25, 0.5]
        )
        yaxis3 = dict(
            visible=True,
            range=[minv, maxv],
            # fixedrange=True,
            spikethickness=1,
            spikedash='dash',
            spikemode='across',
            zeroline=False,
            tick0=close,
            dtick=round(close * 0.01, 2),
            # ticklen=2,
            domain=[0.5, 1]
        )
        xaxis = dict(
            visible=True,
            tickmode='array',
            tickvals=list(range(242)),
            ticktext=HQ.fc.times,
            showticklabels=False,
            showgrid=False,
            fixedrange=True,
            spikethickness=2,
            spikedash='dash',
            spikemode='toaxis'
        )

        yaxis_p = dict(
            visible=True,
            showline=False,
            range=[minv_p, maxv_p],
            fixedrange=True,
            tickmode='linear',
            showgrid=False,
            spikethickness=1,
            spikedash='dash',
            spikemode='toaxis',
            overlaying='y3',
            side='right',
            zeroline=False,
            tick0=0,
            dtick=1,
            # ticklen=2
        )

        fig = tools.make_subplots(rows=3, cols=1, specs=[[{}], [{}], [{}]],
                                  shared_xaxes=True, shared_yaxes=True,
                                  vertical_spacing=0.001)

        fig.append_trace(trace_close, 3, 1)
        fig.append_trace(trace_jj, 3, 1)
        fig.append_trace(trace_p, 3, 1)
        fig.append_trace(bar_to, 2, 1)
        fig.append_trace(trace_fszb, 1, 1)

        fig['layout'].update(
            width=600,
            height=360,
            margin=dict(l=40, r=30, t=10, b=20),
            showlegend=False,
            xaxis=xaxis,
            yaxis=yaxis1,
            yaxis2=yaxis2,
            yaxis3=yaxis3,
            # plot_bgcolor='rgba(0,0,0,0)',
            # paper_bgcolor='rgba(0,0,0,100)',
            yaxis99=yaxis_p
        )
        iplot(fig)


In [ ]:
idx = HQ.codes.index('002486')
plot_fenshi(code_idx=idx, fszb='zhenfu_5m')

In [ ]:
# fc.data[:, 3, 2]

In [ ]:
f = None
for i in range(10):
    if f is None:
        print(f)
        f = 90
print(f)

In [ ]:
cols = ['open', 'close', 'now', 'high', 'low', 'turnover', 'volume', 'zf', 'jj', 'lb', 'zs', 'ds', 'fszb']
data = fc.data[0]
df = pd.DataFrame(data=data, index=HQ.codes, columns=cols)
df.head(5)
# sc.data[:,:,7:13] = np.nan

In [ ]:
print(len(lb_indices), len(zt_indices), len(zs_indices))

In [ ]:
fctimes_1 = fc.times
%store fctimes_1
%store d2a_stock_info_1

In [ ]:
scdata_1 = sc.data
%store scdata_1
fcdata = fc.data
%store fcdata
hqcodes = HQ.codes
%store hqcodes
hqstocks = HQ.stocks
%store hqstocks
# %store -r scdata

In [ ]:
%store